### Interface Conductivity


       P            e                E
     --o------------|----------------o--

       |-- (dx)e- --|---- (dx)e+ ----|
       |------------ dx -------------|

For a steady, no-source, one-dimensional situation in which coductivity varies in a stepwise fashion from on e control volume to the next, the interface-conductivity formulat is given as



$$

k_e = \left(\frac{1-f_e}{k_P} + \frac{f_e}{k_E} \right) ^{-1}

\\
{

  f_e = \frac{(\delta x)_{e+}}{(\delta x)_{e}}
}

$$




In [ ]:
def interface_k(k_P, k_E, dx, dxe):
    f_e = dxe/dx
    return ((1-f_e)/k_P + f_e/k_E)**-1

In [60]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve


node_type_def = np.dtype([
    ('width', '<f4'),
    ('temp', '<f4')
])

T_A = 100
T_B = 500
T_init = 0

width = 0.5
n_nodes = 5

dx = width/n_nodes

k_e = 1000
A = A_e = A_w = 1*10e-3

nodes_list = [
    (0, T_A), # left boundary
    (dx, T_init),
    (dx, T_init),
    (dx, T_init),
    (dx, T_init),
    (dx, T_init),
    (0, T_B) # right boundary
]

nodes = np.array(nodes_list, dtype=node_type_def)

# This gives the distance betweeen the centers of adjacent control volumes
dx_we = nodes[1:]['width']*0.5 + nodes[0:-1]['width']*0.5

a_ws = k_e * A_w / dx_we
a_es = k_e * A_e / dx_we

a_ps = a_ws[1:] + a_es[:-1]

NORMALISER = 1e31

a_ps = np.pad(a_ps, (1, 1), mode='constant', constant_values=(NORMALISER, NORMALISER))

mat = [-a_ws, a_ps, -a_es]

b = np.zeros((7,1))

b[0] = 100*NORMALISER
b[-1] = 500*NORMALISER


A = diags(mat, [-1, 0, 1], format='csr')

x = spsolve(A, b)

print(A.toarray())
print(x)


[[ 1.e+31 -2.e+02  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [-2.e+02  3.e+02 -1.e+02  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00 -1.e+02  2.e+02 -1.e+02  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00 -1.e+02  2.e+02 -1.e+02  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00 -1.e+02  2.e+02 -1.e+02  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00 -1.e+02  3.e+02 -2.e+02]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -2.e+02  1.e+31]]
[100.00000152 140.00000212 220.00000334 300.00000455 380.00000577
 460.00000698 500.00000759]


In [114]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve


node_type_def = np.dtype([
    ('width', '<f4'),
    ('temp', '<f4')
])

T_A = 100
T_B = 500
T_init = 0

width = 0.5
n_nodes = 5

dx = width/n_nodes

k_e = 1000
A = A_e = A_w = 1*10e-3

nodes_list = [
    (0, T_A), # left boundary
    (dx, T_init),
    (dx, T_init),
    (dx, T_init),
    (dx, T_init),
    (dx, T_init),
    (0, T_B) # right boundary
]

nodes = np.array(nodes_list, dtype=node_type_def)

# This gives the distance betweeen the centers of adjacent control volumes
dx_w = nodes[1:-1]['width']*0.5 + nodes[0:-2]['width']*0.5
dx_e = nodes[1:-1]['width']*0.5 + nodes[2:]['width']*0.5

a_ws = k_e * A_w / dx_w
a_es = k_e * A_e / dx_e

a_ps = a_ws + a_es

a_ps = np.pad(a_ps, (1, 1), mode='constant', constant_values=(1, 1))

a_ws = np.pad(a_ws, (0, 1), mode='constant')
a_es = np.pad(a_es, (1, 0), mode='constant')

mat = [-a_ws, a_ps, -a_es]

b = np.zeros((7,1))

b[0] = 100
b[-1] = 500


A = diags(mat, [-1, 0, 1], format='csr')

x = spsolve(A, b)

print(A.toarray())
print(x)

[[   1.    0.    0.    0.    0.    0.    0.]
 [-200.  300. -100.    0.    0.    0.    0.]
 [   0. -100.  200. -100.    0.    0.    0.]
 [   0.    0. -100.  200. -100.    0.    0.]
 [   0.    0.    0. -100.  200. -100.    0.]
 [   0.    0.    0.    0. -100.  300. -200.]
 [   0.    0.    0.    0.    0.    0.    1.]]
[100. 140. 220. 300. 380. 460. 500.]


In [107]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve


node_type_def = np.dtype([
    ('width', '<f4'),
    ('height', '<f4'),
    ('temp', '<f4')
])

T_left = 500
T_right = 500
T_top = 500
T_bottom = 500

T_init = 0

width = 0.5
height = 0.5
nx_nodes = 5
ny_nodes = 5

dx = width/nx_nodes
dy = height/ny_nodes

k_e = k_w = k_n = k_s = 1000
A = A_e = A_w = A_n = A_s = 1*10e-3

nodes_list = [
    # Top
    [ (0, 0, T_top),
    (dx, 0, T_top),
    (dx, 0, T_top),
    (dx, 0, T_top),
    (dx, 0, T_top),
    (dx, 0, T_top),
    (0, 0, T_top) ],

    [ (0, 0, T_left),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (0, 0, T_right) ],

     [ (0, 0, T_left),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (0, 0, T_right) ],

     [ (0, 0, T_left),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (dx, dy, T_init),
    (0, 0, T_right) ],

    # Bottom
    [ (0, 0, T_bottom),
    (dx, 0, T_bottom),
    (dx, 0, T_bottom),
    (dx, 0, T_bottom),
    (dx, 0, T_bottom),
    (dx, 0, T_bottom),
    (0, 0, T_bottom) ],
]

nodes = np.array(nodes_list, dtype=node_type_def)

a_Ws = k_w * dy / (nodes[1:-1, 1:-1]['width']*0.5 + nodes[1:-1, 0:-2]['width']*0.5)
a_Es = k_e * dy / (nodes[1:-1, 1:-1]['width']*0.5 + nodes[1:-1, 2:]['width']*0.5)

a_Ns = k_n * dx / (nodes[1:-1, 1:-1]['height']*0.5 + nodes[0:-2, 1:-1]['height']*0.5)
a_Ss = k_s * dx / (nodes[1:-1, 1:-1]['height']*0.5 + nodes[2:, 1:-1]['height']*0.5)




(array([[2000., 2000., 2000., 2000., 2000.],
        [1000., 1000., 1000., 1000., 1000.],
        [1000., 1000., 1000., 1000., 1000.]], dtype=float32),
 array([[1000., 1000., 1000., 1000., 1000.],
        [1000., 1000., 1000., 1000., 1000.],
        [2000., 2000., 2000., 2000., 2000.]], dtype=float32))

In [106]:
(nodes[1:-1, 1:-1]['width']*0.5 + nodes[0:-2, 1:-1]['width']*0.5)

array([[0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1]], dtype=float32)